# [M_00] PROTOKÓŁ: INICJACJA INFRASTRUKTURY

![Status](https://img.shields.io/badge/STATUS-W_BUDOWIE-8B0000?style=for-the-badge)
![Python](https://img.shields.io/badge/PYTHON-3.12-003366?style=for-the-badge&logo=python&logoColor=white)
![Framework](https://img.shields.io/badge/FRAMEWORK-PYDANTIC--AI_v1.39.0-006400?style=for-the-badge)
![Manager](https://img.shields.io/badge/MANAGER-UV-800020?style=for-the-badge)

**PROJEKT:** OMNI-OPERATOR-V1  
**NOTATNIK:** 00_INFRASTRUKTURA  
**LOKALIZACJA:** `notebooks/`

**CEL:** Przygotowanie fundamentów systemu w katalogu głównym projektu.

## KROK 0: Ustawienie kontekstu ścieżek
Ponieważ ten notatnik znajduje się w podfolderze `notebooks/`, musimy upewnić się, że skrypty będą generować pliki w folderze nadrzędnym (root projektu).

In [1]:
import os

# Sprawdzenie aktualnej lokalizacji i przejście do folderu nadrzędnego (root)
current_dir = os.getcwd()
if current_dir.endswith("notebooks"):
    os.chdir("..")
    print(f"LOG: Zmieniono katalog roboczy na: {os.getcwd()}")
else:
    print(f"LOG: Katalog roboczy to już root: {os.getcwd()}")

LOG: Zmieniono katalog roboczy na: c:\Users\takze\OneDrive\Pulpit\project\omni-operator-v1


## KROK 1: Konfiguracja `pyproject.toml`
Tworzymy definicję projektu w trybie aplikacji (`package = false`), aby uniknąć problemów z budowaniem paczek przez Hatchling.

In [2]:
pyproject_content = """
[project]
name = "OMNI-OPERATOR-V1"
version = "0.1.0"
description = "Autonomiczna fabryka dystrybucji treści - Flagowiec Kuźni Operatorów"
readme = "README.md"
requires-python = ">=3.12"
dependencies = [
    "pydantic-ai==1.39.0",
    "google-generativeai>=0.8.6",
    "fastapi>=0.128.0",
    "uvicorn>=0.40.0",
    "qdrant-client>=1.16.2",
    "langfuse>=3.11.2",
    "moviepy>=2.2.1",
    "python-dotenv>=1.2.1",
    "pydantic-settings>=2.4.0",
    "torch",
    "torchvision",
    "torchaudio",
]

[tool.uv]
managed = true
package = false
"""

with open("pyproject.toml", "w", encoding="utf-8") as f:
    f.write(pyproject_content.strip())

print("LOG: Plik pyproject.toml wygenerowany w katalogu głównym.")

LOG: Plik pyproject.toml wygenerowany w katalogu głównym.


## KROK 2: Struktura katalogów
Inicjalizujemy architekturę folderów `src/` oraz `docker/`.

In [3]:
folders = [
    "src/agents",
    "src/core",
    "src/services",
    "src/api",
    "docker",
    "notebooks"
]

for folder in folders:
    os.makedirs(folder, exist_ok=True)
    # Dodajemy __init__.py tylko do folderów w src/
    if folder.startswith("src"):
        init_file = os.path.join(folder, "__init__.py")
        if not os.path.exists(init_file):
            with open(init_file, "w") as f:
                pass

print("LOG: Struktura katalogów src/ oraz docker/ jest gotowa.")

LOG: Struktura katalogów src/ oraz docker/ jest gotowa.


## KROK 3: Infrastruktura Docker
Definiujemy kontenery dla bazy Qdrant oraz systemu monitoringu Langfuse.

In [4]:
docker_compose_content = """
services:
  qdrant:
    image: qdrant/qdrant:v1.12.0
    container_name: omni_qdrant
    ports:
      - "6333:6333"
    volumes:
      - ./qdrant_data:/qdrant/storage

  db:
    image: postgres:16-alpine
    container_name: omni_db
    environment:
      POSTGRES_USER: postgres
      POSTGRES_PASSWORD: password
      POSTGRES_DB: langfuse
    ports:
      - "5432:5432"
    volumes:
      - ./postgres_data:/var/lib/postgresql/data

  langfuse:
    image: langfuse/langfuse:2
    container_name: omni_langfuse
    depends_on:
      - db
    ports:
      - "3000:3000"
    environment:
      - DATABASE_URL=postgresql://postgres:password@db:5432/langfuse
      - NEXTAUTH_URL=http://localhost:3000
      - NEXTAUTH_SECRET=mysecret
      - SALT=mysalt
"""

with open("docker-compose.yml", "w", encoding="utf-8") as f:
    f.write(docker_compose_content.strip())

print("LOG: Plik docker-compose.yml wygenerowany.")

LOG: Plik docker-compose.yml wygenerowany.


## KROK 4: Zarządzanie konfiguracją
Tworzymy `src/core/config.py` oraz szablon `.env.example`.

In [5]:
# .env.example
env_example = """
GEMINI_API_KEY=AIza...
LANGFUSE_PUBLIC_KEY=pk-lf-...
LANGFUSE_SECRET_KEY=sk-lf-...
LANGFUSE_HOST=http://localhost:3000
QDRANT_URL=http://localhost:6333
""".strip()

with open(".env.example", "w") as f:
    f.write(env_example)

# src/core/config.py
config_py = """
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field

class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_file=".env", extra="ignore")

    gemini_api_key: str = Field(alias="GEMINI_API_KEY")
    langfuse_public_key: str = Field(alias="LANGFUSE_PUBLIC_KEY")
    langfuse_secret_key: str = Field(alias="LANGFUSE_SECRET_KEY")
    langfuse_host: str = Field(default="http://localhost:3000", alias="LANGFUSE_HOST")
    qdrant_url: str = Field(default="http://localhost:6333", alias="QDRANT_URL")

settings = Settings()
"""

with open("src/core/config.py", "w", encoding="utf-8") as f:
    f.write(config_py.strip())

print("LOG: Konfiguracja src/core/config.py wdrożona.")

LOG: Konfiguracja src/core/config.py wdrożona.


## KROK 5: .gitignore i Synchronizacja `uv`
Na koniec zabezpieczamy repozytorium i instalujemy biblioteki.

In [ ]:
gitignore = """
.venv/
__pycache__/
.env
qdrant_data/
postgres_data/
*.mp4
*.srt
uv.lock
"""

with open(".gitignore", "w") as f:
    f.write(gitignore.strip())

print("LOG: .gitignore utworzony. Synchronizuję środowisko uv...")

# Wywołanie systemowe uv
!uv sync

## STATUS: INFRASTRUKTURA GOTOWA

Projekt **OMNI-OPERATOR-V1** posiada teraz kompletną bazę operacyjną.

**Co zostało zrobione:**
1. Przejście do root projektu z poziomu `notebooks/`.
2. Generacja `pyproject.toml` (Python 3.12 + uv).
3. Setup `src/` i `docker-compose.yml`.
4. Implementacja `src/core/config.py`.
5. Synchronizacja bibliotek.

**Następny etap:** Pierwszy commit i budowa Agenta Analityka.